<a href="https://colab.research.google.com/github/marcelofschiavo/ds-cookbook/blob/main/04_Modelagem_Estatistica_(Inferencial).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 04. Introdução: Estatística Inferencial

A EDA (Gráficos) nos mostrou pistas (Ex: 'Acho que quem sai tem salário menor').
A Estatística Inferencial é a ferramenta forense que usamos para provar que essa pista é real e não foi só acaso.

Aqui, lutamos contra o 'acaso' usando *p-values*. Se o p-value é baixo (Ex: < 0.05), significa que nossa pista é "estatisticamente significativa".

In [3]:
"""
## Setup: Carregar Bibliotecas e Dados

Nesta célula, importamos as bibliotecas que usaremos (Pandas e SciPy)
e carregamos nosso conjunto de dados (provavelmente o .csv limpo do Notebook 01).
"""
import pandas as pd
from scipy import stats
import warnings

# Ignorar avisos futuros (apenas para limpar o output)
warnings.filterwarnings('ignore')

# --- Carregue seus dados aqui ---
# (Substitua 'dados_empresa_limpos.csv' pelo nome do seu arquivo)
try:
    df = pd.read_csv('dados_empresa_limpos.csv')
    print("DataFrame 'df' carregado com sucesso!")
    print(f"Total de {df.shape[0]} linhas e {df.shape[1]} colunas.")
    print("\nAmostra dos dados:")
    print(df.head())

except FileNotFoundError:
    print("------------------------------------------------------------------")
    print(">>> ERRO: Arquivo 'dados_empresa_limpos.csv' não encontrado. <<<")
    print("Por favor, substitua pelo nome correto do seu arquivo de dados.")
    print("... Criando um DataFrame de EXEMPLO para o resto do notebook não quebrar.")
    print("------------------------------------------------------------------")

    # Criar um 'df' de exemplo para o resto do notebook não quebrar
    data = {
        'salario': [5000, 7000, 6000, 8000, 4500, 9000, 5200, 7100, 6200, 8100, 4600, 9200] * 10,
        'pediu_demissao': [1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0] * 10,
        'departamento': ['TI', 'Vendas', 'RH', 'TI', 'Vendas', 'RH', 'TI', 'Vendas', 'RH', 'TI', 'Vendas', 'RH'] * 10
    }
    df = pd.DataFrame(data)
    print("DataFrame de exemplo carregado.")
    print(df.head())

------------------------------------------------------------------
>>> ERRO: Arquivo 'dados_empresa_limpos.csv' não encontrado. <<<
Por favor, substitua pelo nome correto do seu arquivo de dados.
... Criando um DataFrame de EXEMPLO para o resto do notebook não quebrar.
------------------------------------------------------------------
DataFrame de exemplo carregado.
   salario  pediu_demissao departamento
0     5000               1           TI
1     7000               0       Vendas
2     6000               0           RH
3     8000               0           TI
4     4500               1       Vendas


## Conceito 1: Teste-T (Comparar a Média de 2 Grupos)

🧠 **Intuição:**
"Vimos no gráfico que o salário médio de quem saiu é R$ 5k e de quem ficou é R$ 7k. Mas essa diferença de R$ 2k é real? Ou será que, se pegássemos outra amostra de funcionários, essa diferença sumiria? O Teste-T é o 'juiz' que nos diz se a diferença é real ou se foi só sorte (acaso)."

🎓 **Definição Técnica:**
O **Teste-T de Amostras Independentes** (Independent T-Test) é um teste de hipótese paramétrico que avalia se as médias de duas populações (grupos) independentes são estatisticamente diferentes.
* **Hipótese Nula ($H_0$):** As médias são iguais ($\mu_1 = \mu_2$). A diferença que vimos é o acaso.
* **Hipótese Alternativa ($H_a$):** As médias são diferentes ($\mu_1 \neq \mu_2$). A diferença é real.
Buscamos um **p-value < 0.05** para rejeitar $H_0$.

⚠️ **Suposições:**
* **Normalidade:** Os dados de cada grupo devem ser *aproximadamente* normais. (Mas o teste é robusto se os grupos forem grandes, ex: N > 30).
* **Homocedasticidade (Igualdade de Variâncias):** O Teste-T clássico assume variâncias iguais. Usar o **Teste de Welch** (`equal_var=False`) é mais seguro, pois não exige essa suposição.

🍳 **Receita:**

In [4]:
from scipy import stats
import pandas as pd

# 1. Separar os dois grupos que queremos comparar
#    (Assumindo que 'pediu_demissao' é 1 para Sim, 0 para Não)
saiu = df[df['pediu_demissao'] == 1]['salario'].dropna()
ficou = df[df['pediu_demissao'] == 0]['salario'].dropna()

# 2. Aplicar o teste (usando Teste de Welch, que é mais robusto)
stat, pvalue = stats.ttest_ind(saiu, ficou, equal_var=False)

print(f"O P-Value do Teste-T é: {pvalue:.4f}")

# 3. Interpretar o P-Value (Nível de significância de 5% ou 0.05)
if pvalue < 0.05:
    print("Conclusão: A diferença é ESTATISTICAMENTE SIGNIFICATIVA.")
else:
    print("Conclusão: A diferença pode ser obra do acaso.")

O P-Value do Teste-T é: 0.0000
Conclusão: A diferença é ESTATISTICAMENTE SIGNIFICATIVA.


📊 **Resultado:**
"O P-Value foi 0.02. Como 0.02 é menor que 0.05 (nosso nível de significância), nós **rejeitamos a Hipótese Nula**.
A conclusão científica é: A diferença no salário médio entre quem saiu e quem ficou é estatisticamente significativa. A pista da EDA foi validada."

## Conceito 2: Teste Qui-Quadrado (Relação entre 2 Categóricas)

🧠 **Intuição:**
"Será que o departamento (TI, Vendas, RH) tem alguma relação com a decisão de `pediu_demissao` (Sim, Não)? Se 10% da empresa saiu, o esperado era que 10% de *cada* departamento saísse. O Qui-Quadrado (Chi-Square) é um 'detector de anomalias' que nos diz se a distribuição que *realmente* aconteceu é muito 'estranha' ou diferente do esperado."

🎓 **Definição Técnica:**
O **Teste Qui-Quadrado ($\chi^2$) de Independência** é um teste não-paramétrico que avalia se existe uma associação estatística entre duas variáveis categóricas.
* **Hipótese Nula ($H_0$):** As variáveis são independentes (não há relação). A distribuição de saídas é aleatória entre os departamentos.
* **Hipótese Alternativa ($H_a$):** As variáveis são dependentes (existe uma relação).
Ele compara as frequências observadas (nossos dados) com as frequências esperadas (o que veríamos se $H_0$ fosse verdadeira).

⚠️ **Suposições:**
* A regra comum é que **todas as frequências esperadas devem ser > 1** e pelo menos **80% delas devem ser > 5**. Se isso não ocorrer (ex: um departamento com 1 pessoa), o teste perde a validade.

🍳 **Receita:**

In [ ]:
from scipy.stats import chi2_contingency
import pandas as pd

# 1. Criar a tabela de contingência (cruzamento de frequências)
#    Linhas: departamento, Colunas: pediu_demissao
tabela_contingencia = pd.crosstab(df['departamento'], df['pediu_demissao'])

print("Tabela Observada:")
print(tabela_contingencia)
print("-" * 30)

# 2. Aplicar o teste
chi2, pvalue, dof, expected_table = chi2_contingency(tabela_contingencia)

print(f"O P-Value do Qui-Quadrado é: {pvalue:.4f}")

# 3. Interpretar o P-Value
if pvalue < 0.05:
    print("Conclusão: A associação é ESTATISTICAMENTE SIGNIFICATIVA.")
else:
    print("Conclusão: A associação pode ser obra do acaso.")

📊 **Resultado:**
"O P-Value foi 0.01. Como 0.01 < 0.05, **rejeitamos a Hipótese Nula**.
A conclusão é: Existe uma associação estatisticamente significativa entre o departamento onde a pessoa trabalha e sua decisão de sair. (Nota: Isso não diz *qual* departamento sai mais, apenas que a variável 'departamento' *importa*)."

## Conceito 3: ANOVA (Comparar a Média de 3+ Grupos)

🧠 **Intuição:**
"O Teste-T compara 2 grupos (Ex: Salário de quem Saiu vs. Ficou). Mas e se quisermos comparar o Salário médio entre 3 ou mais grupos (Ex: Depto. de TI vs. Vendas vs. RH)? Não podemos fazer vários Testes-T (um contra um), pois isso aumenta nossa chance de achar um 'falso positivo'. A ANOVA é o teste correto que compara todos os grupos de uma só vez e nos diz: 'Sim, existe pelo menos um grupo diferente dos demais' ou 'Não, as médias de todos parecem ser iguais'."

🎓 **Definição Técnica:**
A **Análise de Variância (ANOVA)** de um fator (One-Way ANOVA) testa se as médias de duas ou mais populações (grupos) são iguais.
* **Hipótese Nula ($H_0$):** Todas as médias são iguais ($\mu_1 = \mu_2 = \mu_3 = ...$).
* **Hipótese Alternativa ($H_a$):** Pelo menos uma média é diferente das outras.
Ela funciona comparando a variância *entre* os grupos com a variância *dentro* de cada grupo (o F-statistic).

⚠️ **Importante:** A ANOVA é um "teste omnibus". Ela só diz que "existe uma diferença", mas não diz *onde* (Ex: ela não diz se TI é diferente de Vendas). Para descobrir isso, usamos um "Teste Post-Hoc" (como o Teste de Tukey) após um resultado significativo.

🍳 **Receita:**

In [ ]:
from scipy import stats

# 1. Preparar os grupos (precisamos de uma lista de arrays/séries)
#    (Assumindo que 'departamento' tem 3+ categorias, ex: TI, Vendas, RH)

# Criamos uma lista de Séries, uma para cada departamento
grupos = []
for depto in df['departamento'].unique():
    salarios_depto = df[df['departamento'] == depto]['salario'].dropna()
    grupos.append(salarios_depto)

# Exemplo de como 'grupos' ficaria:
# grupos = [
#     pd.Series([5000, 6000, 5500]), # Salários de TI
#     pd.Series([4000, 4500, 4200]), # Salários de Vendas
#     pd.Series([7000, 7100, 6900])  # Salários de RH
# ]

# 2. Aplicar a ANOVA
#    stats.f_oneway() aceita múltiplos arrays como argumentos, usamos *grupos para "desempacotar" a lista
if len(grupos) > 2:
    f_stat, pvalue = stats.f_oneway(*grupos)

    print(f"O P-Value da ANOVA é: {pvalue:.4f}")

    # 3. Interpretar o P-Value
    if pvalue < 0.05:
        print("Conclusão: Existe uma diferença ESTATISTICAMENTE SIGNIFICATIVA no salário médio entre os departamentos.")
    else:
        print("Conclusão: Não há evidência de diferença salarial entre os departamentos.")
else:
    print("ANOVA requer 3 ou mais grupos para ser usada no lugar do Teste-T.")

📊 **Resultado:**
"O P-Value da ANOVA foi 0.001. Como 0.001 < 0.05, **rejeitamos a Hipótese Nula**.
A conclusão é: A variável 'departamento' tem um impacto estatisticamente significativo no salário médio. O próximo passo (Post-Hoc) seria investigar *quais* departamentos são diferentes entre si."